# Description

**TODO**

# Modules

In [1]:
library(reticulate)
pd <- import("pandas")

# Settings

In [2]:
# reference panel
REFERENCE_PANEL <- "GTEX_V8"
# REFERENCE_PANEL = "1000G"

# prediction models
## mashr
EQTL_MODEL <- "MASHR"

# ## elastic net
# EQTL_MODEL = "ELASTIC_NET"

chromosome <- NULL

In [3]:
# Parameters
chromosome = 6
REFERENCE_PANEL = "GTEX_V8"
EQTL_MODEL = "MASHR"


In [4]:
paste0("Using reference panel: ", REFERENCE_PANEL)

[1] "Using reference panel: GTEX_V8"

In [5]:
paste0("Using eQTL model: ", EQTL_MODEL)

[1] "Using eQTL model: MASHR"

In [6]:
# chromosome must be provided as parameter
stopifnot(!is.null(chromosome))

# Paths

In [7]:
GENE_CORRS_DIR <- Sys.getenv("PHENOPLIER_PHENOMEXCAN_LD_BLOCKS_GENE_CORRS_DIR")
IRdisplay::display(GENE_CORRS_DIR)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs"

In [8]:
INPUT_DIR <- file.path(GENE_CORRS_DIR, tolower(REFERENCE_PANEL), tolower(EQTL_MODEL), "by_chr")
IRdisplay::display(INPUT_DIR)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr"

In [9]:
INPUT_FILE <- file.path(INPUT_DIR, paste0("gene_corrs-chr", chromosome, ".pkl"))
IRdisplay::display(INPUT_FILE)
stopifnot(file.exists(INPUT_FILE))

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/gene_corrs-chr6.pkl"

In [10]:
OUTPUT_DIR <- file.path(INPUT_DIR, "corrected_positive_definite")
IRdisplay::display(OUTPUT_DIR)
dir.create(OUTPUT_DIR, recursive = TRUE, showWarnings = FALSE)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/corrected_positive_definite"

In [11]:
OUTPUT_FILE <- file.path(OUTPUT_DIR, paste0("gene_corrs-chr", chromosome, ".pkl"))
IRdisplay::display(OUTPUT_FILE)
if (file.exists(OUTPUT_FILE)) {
  IRdisplay::display("Output file exists, it will be overwritten")
}

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/corrected_positive_definite/gene_corrs-chr6.pkl"

[1] "Output file exists, it will be overwritten"

# Functions

In [12]:
# taken from https://www.r-bloggers.com/2013/08/correcting-a-pseudo-correlation-matrix-to-be-positive-semidefinite/
# TODO: add documentation
CorrectCM <- function(CM, p = 0) {
  n <- dim(var(CM))[1L]
  E <- eigen(CM)
  CM1 <- E$vectors %*% tcrossprod(diag(pmax(E$values, p), n), E$vectors)
  Balance <- diag(1 / sqrt(diag(CM1)))
  CM2 <- Balance %*% CM1 %*% Balance
  return(CM2)
}

In [13]:
is_positive_definite <- function(data) {
  eigenvalues <- eigen(data)$values
  nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]

  if (length(nonpositive_eigenvalues) > 0) {
    IRdisplay::display("We need to correct the data and make the matrix positive definite")
    return(FALSE)
  } else {
    IRdisplay::display("Matrix is already positive definite!")
    return(TRUE)
  }
}

# Load data

In [14]:
gene_corrs <- pd$read_pickle(INPUT_FILE)

In [15]:
dim(gene_corrs)

[1] 339 339

In [16]:
head(gene_corrs[1:10, 1:10])

,ENSG00000137265,ENSG00000112685,ENSG00000112699,ENSG00000124570,ENSG00000137275,ENSG00000137267,ENSG00000112739,ENSG00000153046,ENSG00000145982,ENSG00000124491
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000137265,1.000000000,-0.27809026,-0.177070395,0.09827798,0.009274999,0.01900055,-0.11003145,0.01663026,0.09539731,0.00000000
ENSG00000112685,-0.278090259,1.00000000,-0.101520677,-0.08038313,0.240844468,0.10141871,0.18877883,0.01610206,0.01665490,0.00000000
ENSG00000112699,-0.177070395,-0.10152068,1.000000000,-0.06915009,0.007904048,-0.07162800,-0.12961447,0.01359337,-0.05033236,0.08622933
ENSG00000124570,0.098277977,-0.08038313,-0.069150087,1.00000000,0.193548169,0.07951803,-0.03731162,-0.02430548,0.02961757,-0.01736038
ENSG00000137275,0.009274999,0.24084447,0.007904048,0.19354817,1.000000000,-0.06516748,-0.01667263,-0.16261081,-0.07714674,-0.07548545
ENSG00000137267,0.019000552,0.10141871,-0.071627999,0.07951803,-0.065167480,1.00000000,-0.09737381,0.07528770,-0.04259903,-0.02268582


# Check positive definiteness

In [17]:
is_positive_definite(gene_corrs)

[1] "We need to correct the data and make the matrix positive definite"

[1] FALSE

In [18]:
# see eigenvalues
eigenvalues <- eigen(gene_corrs)$values

In [19]:
nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]
IRdisplay::display(length(nonpositive_eigenvalues))
IRdisplay::display(nonpositive_eigenvalues)

[1] 28

[1] -0.01052232 -0.02290824 -0.04459119 -0.07299248 -0.11270032 -0.14116372
 [7] -0.15416387 -0.18261129 -0.20311034 -0.26794375 -0.27928383 -0.29990638
[13] -0.33247060 -0.37744846 -0.41995030 -0.45920901 -0.50907528 -0.60393175
[19] -0.68264466 -0.72988847 -0.78702869 -0.83762769 -1.03015203 -1.14367353
[25] -1.29273719 -1.39226892 -1.77995173 -1.96357161

In [20]:
if (length(eigenvalues[eigenvalues <= 0]) == 0) { quit() }

# Make matrix positive definite if needed

In [21]:
gene_corrs_corrected <- CorrectCM(gene_corrs, 1)

In [22]:
dimnames(gene_corrs_corrected)[[1]] <- rownames(gene_corrs)

In [23]:
dimnames(gene_corrs_corrected)[[2]] <- colnames(gene_corrs)

In [24]:
gene_corrs_corrected <- as.data.frame(gene_corrs_corrected)

In [25]:
dim(gene_corrs_corrected)

[1] 339 339

# Check positive definiteness of corrected matrix

In [26]:
is_positive_definite(gene_corrs_corrected)

[1] "Matrix is already positive definite!"

[1] TRUE

In [27]:
# see eigenvalues
eigenvalues <- eigen(gene_corrs_corrected)$values

In [28]:
nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]
IRdisplay::display(length(nonpositive_eigenvalues))
IRdisplay::display(nonpositive_eigenvalues)

[1] 0

numeric(0)

In [29]:
stopifnot(length(eigenvalues[eigenvalues <= 0]) == 0)

In [30]:
# quick and visual comparison of the two matrices
IRdisplay::display(head(gene_corrs[1:10, 1:10]))
IRdisplay::display(head(gene_corrs_corrected[1:10, 1:10]))

,ENSG00000137265,ENSG00000112685,ENSG00000112699,ENSG00000124570,ENSG00000137275,ENSG00000137267,ENSG00000112739,ENSG00000153046,ENSG00000145982,ENSG00000124491
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000137265,1.000000000,-0.27809026,-0.177070395,0.09827798,0.009274999,0.01900055,-0.11003145,0.01663026,0.09539731,0.00000000
ENSG00000112685,-0.278090259,1.00000000,-0.101520677,-0.08038313,0.240844468,0.10141871,0.18877883,0.01610206,0.01665490,0.00000000
ENSG00000112699,-0.177070395,-0.10152068,1.000000000,-0.06915009,0.007904048,-0.07162800,-0.12961447,0.01359337,-0.05033236,0.08622933
ENSG00000124570,0.098277977,-0.08038313,-0.069150087,1.00000000,0.193548169,0.07951803,-0.03731162,-0.02430548,0.02961757,-0.01736038
ENSG00000137275,0.009274999,0.24084447,0.007904048,0.19354817,1.000000000,-0.06516748,-0.01667263,-0.16261081,-0.07714674,-0.07548545
ENSG00000137267,0.019000552,0.10141871,-0.071627999,0.07951803,-0.065167480,1.00000000,-0.09737381,0.07528770,-0.04259903,-0.02268582


,ENSG00000137265,ENSG00000112685,ENSG00000112699,ENSG00000124570,ENSG00000137275,ENSG00000137267,ENSG00000112739,ENSG00000153046,ENSG00000145982,ENSG00000124491
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000137265,1.00000000,-0.12027295,-0.05288600,0.07110942,-0.02861347,0.01064101,-0.07314275,0.008969699,0.05993289,-0.01962484
ENSG00000112685,-0.12027295,1.00000000,-0.03003273,-0.01723853,0.08190161,0.02057133,0.10630067,-0.010750120,-0.03766669,-0.01625513
ENSG00000112699,-0.05288600,-0.03003273,1.00000000,-0.04296567,-0.01938090,-0.03126121,-0.04425201,-0.007158365,-0.03680762,0.04300042
ENSG00000124570,0.07110942,-0.01723853,-0.04296567,1.00000000,0.06988945,0.03786483,-0.03348792,-0.034760124,0.01419730,-0.01727914
ENSG00000137275,-0.02861347,0.08190161,-0.01938090,0.06988945,1.00000000,-0.01257234,0.03625025,-0.079703331,-0.01963157,-0.03767028
ENSG00000137267,0.01064101,0.02057133,-0.03126121,0.03786483,-0.01257234,1.00000000,-0.03837867,0.033169726,-0.02585493,-0.02349812


Both matrices should "look" similar. We are not interested in perfectly accurate correlation values (they are already inaccurate).

# Save

In [31]:
py_save_object(gene_corrs_corrected, OUTPUT_FILE)